In [1]:
# Import libraries to be used

# Warning messages display
## import warnings
## warnings.filterwarnings(action='ignore') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Timing
import time

# Memory monitoring
%load_ext memory_profiler
### Use '%memit' to check at each point

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
## pd.set_option('display.max_rows', None) # It greatly slows down the output display and freezes the kernel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [2]:
t0 = time.perf_counter() 

In [3]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

Running on Windows.
root path	 C:\Users\turge\CompartidoVM\0.TFM


Additional information on each column meaning can be found [here](https://www.ncei.noaa.gov/data/local-climatological-data/doc/LCD_documentation.pLCD).

___

# Get the data

## LCD individual file (2019)

### Import file

#### Define file path

In [4]:
csv_files_path = os.path.join(root,
                              "Output_Data",
                              "NOAA",
                              "LCD_files")
csv_files_path

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files'

In [5]:
individualLCLCDileName = "ABE_14737.csv"
individualLCLCDileNamePath = os.path.join(csv_files_path, individualLCLCDileName)
individualLCLCDileNamePath

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABE_14737.csv'

#### Select the interesting columns

In [6]:
cols = [
 'STATION', # Station ID (VMO+WBAN)
 'DATE', # Date: YYYY-MM-DD format / Time: 24-hour clock in local standard time (LST)
#  'LATITUDE',
#  'LONGITUDE',
#  'ELEVATION',
#  'NAME',
#  'REPORT_TYPE',
#  'SOURCE',
 'HourlyAltimeterSetting', # Atmospheric pressure reduced to sea level using temperature profile of the “standard” atmosphere
#  'HourlyDewPointTemperature',
 'HourlyDryBulbTemperature', # Commonly used as the standard air temperature reported (given in whole degrees Fahrenheit)
 'HourlyPrecipitation', # Amount of precipitation in inches to hundredths over the past hour (“T” indicates a trace amount)
#  'HourlyPresentWeatherType',
#  'HourlyPressureChange',
#  'HourlyPressureTendency',
 'HourlyRelativeHumidity', # Relative humidity given to the nearest whole percentage
 'HourlySkyConditions', # A report of each cloud layer (up to 3).Each layer given in the following format: ccc:ll-xxx where:
                         ## ccc = Coverage: CLR (clear sky), FEW (few clouds), SCT (scattered clouds), BKN (broken clouds),
                         ##                 OVC (overcast), VV (obscured sky), 10 (partially obscured sky) → Last layer used (*)
                         ##  ll = Layer amount given in eighths (aka “oktas”) → Not used
                         ## xxx = Cloud base height at lowest point of layer → Not used   
                             ## (*) NOTE: Since up to 3 cloud layers can be reported, the full state of the sky 
                             ##           can best be determined by the contraction given for the last layer
#  'HourlySeaLevelPressure',
#  'HourlyStationPressure',
 'HourlyVisibility', # Horizontal distance an object can be seen and identified given in whole miles
#  'HourlyWetBulbTemperature',
#  'HourlyWindDirection', # wind direction/speed/gust variables would be useful in case RWY Magnetic Heading was also available
#  'HourlyWindGustSpeed',   # without it, there is no use for it. Strong crosswinds can greatly penalize A/C performance;
#  'HourlyWindSpeed',       # however, RWYs are normally designed so that headwind prevails to improve A/C performance
#  'Sunrise',
#  'Sunset',
#  'DailyAverageDewPointTemperature',
#  'DailyAverageDryBulbTemperature',
#  'DailyAverageRelativeHumidity',
#  'DailyAverageSeaLevelPressure',
#  'DailyAverageStationPressure',
#  'DailyAverageWetBulbTemperature',
#  'DailyAverageWindSpeed',
#  'DailyCoolingDegreeDays',
#  'DailyDepartureFromNormalAverageTemperature',
#  'DailyHeatingDegreeDays',
#  'DailyMaximumDryBulbTemperature',
#  'DailyMinimumDryBulbTemperature',
#  'DailyPeakWindDirection',
#  'DailyPeakWindSpeed',
#  'DailyPrecipitation',
#  'DailySnowDepth',
#  'DailySnowfall',
#  'DailySustainedWindDirection',
#  'DailySustainedWindSpeed',
#  'DailyWeather',
#  'MonthlyAverageRH',
#  'MonthlyDaysWithGT001Precip',
#  'MonthlyDaysWithGT010Precip',
#  'MonthlyDaysWithGT32Temp',
#  'MonthlyDaysWithGT90Temp',
#  'MonthlyDaysWithLT0Temp',
#  'MonthlyDaysWithLT32Temp',
#  'MonthlyDepartureFromNormalAverageTemperature',
#  'MonthlyDepartureFromNormalCoolingDegreeDays',
#  'MonthlyDepartureFromNormalHeatingDegreeDays',
#  'MonthlyDepartureFromNormalMaximumTemperature',
#  'MonthlyDepartureFromNormalMinimumTemperature',
#  'MonthlyDepartureFromNormalPrecipitation',
#  'MonthlyDewpointTemperature',
#  'MonthlyGreatestPrecip',
#  'MonthlyGreatestPrecipDate',
#  'MonthlyGreatestSnowDepth',
#  'MonthlyGreatestSnowDepthDate',
#  'MonthlyGreatestSnowfall',
#  'MonthlyGreatestSnowfallDate',
#  'MonthlyMaxSeaLevelPressureValue',
#  'MonthlyMaxSeaLevelPressureValueDate',
#  'MonthlyMaxSeaLevelPressureValueTime',
#  'MonthlyMaximumTemperature',
#  'MonthlyMeanTemperature',
#  'MonthlyMinSeaLevelPressureValue',
#  'MonthlyMinSeaLevelPressureValueDate',
#  'MonthlyMinSeaLevelPressureValueTime',
#  'MonthlyMinimumTemperature',
#  'MonthlySeaLevelPressure',
#  'MonthlyStationPressure',
#  'MonthlyTotalLiquidPrecipitation',
#  'MonthlyTotalSnowfall',
#  'MonthlyWetBulb',
#  'AWND',
#  'CDSD',
#  'CLDD',
#  'DSNW',
#  'HDSD',
#  'HTDD',
#  'NormalsCoolingDegreeDay',
#  'NormalsHeatingDegreeDay',
#  'ShortDurationEndDate005',
#  'ShortDurationEndDate010',
#  'ShortDurationEndDate015',
#  'ShortDurationEndDate020',
#  'ShortDurationEndDate030',
#  'ShortDurationEndDate045',
#  'ShortDurationEndDate060',
#  'ShortDurationEndDate080',
#  'ShortDurationEndDate100',
#  'ShortDurationEndDate120',
#  'ShortDurationEndDate150',
#  'ShortDurationEndDate180',
#  'ShortDurationPrecipitationValue005',
#  'ShortDurationPrecipitationValue010',
#  'ShortDurationPrecipitationValue015',
#  'ShortDurationPrecipitationValue020',
#  'ShortDurationPrecipitationValue030',
#  'ShortDurationPrecipitationValue045',
#  'ShortDurationPrecipitationValue060',
#  'ShortDurationPrecipitationValue080',
#  'ShortDurationPrecipitationValue100',
#  'ShortDurationPrecipitationValue120',
#  'ShortDurationPrecipitationValue150',
#  'ShortDurationPrecipitationValue180',
 'REM', # Surface Weather Observations & Reports (METAR)
#  'BackupDirection',
#  'BackupDistance',
#  'BackupDistanceUnit',
#  'BackupElements',
#  'BackupElevation',
#  'BackupEquipment',
#  'BackupLatitude',
#  'BackupLongitude',
#  'BackupName',
#  'WindEquipmentChangeDate'
]

Selected columns:

- `STATION` → Station ID (VMO+WBAN)  
- `DATE` → Date: YYYY-MM-DD format / Time: 24-hour clock in local standard time (LST)  
- `HourlyAltimeterSetting` → Atmospheric pressure reduced to sea level using temperature profile of the “standard” atmosphere  
- `HourlyDryBulbTemperature` → Commonly used as the standard air temperature reported (given in whole degrees Fahrenheit)  
- `HourlyPrecipitation` → Amount of precipitation in inches to hundredths over the past hour (“T” indicates a trace amount)   
- `HourlyRelativeHumidity` → Relative humidity given to the nearest whole percentage  
- `HourlySkyConditions` → A report of each cloud layer (up to 3).  
Each layer given in the following format: `ccc:ll-xxx` where:
    - ccc = Coverage:
        - CLR (clear sky)
        - FEW (few clouds)
        - SCT (scattered clouds)
        - BKN (broken clouds)
        - OVC (overcast)
        - VV (obscured sky)
        - 10 (partially obscured sky) → **Only last layer used (*)**
    - ll = Layer amount given in eighths (aka “oktas”) → *Not used*
    - xxx = Cloud base height at lowest point of layer → *Not used*  
    (*) NOTE: Since up to 3 cloud layers can be reported, the full state of the sky can best be determined by the contraction given for the last layer.
- `HourlyVisibility` → Horizontal distance an object can be seen and identified given in whole miles
- `REM` → Surface Weather Observations & Reports (METAR)
    
Additional note for **wind**:  
- `HourlyWindDirection`
- `HourlyWindGustSpeed`
- `HourlyWindSpeed`  
Wind direction/speed/gust variables would be useful in case RWY Magnetic Heading was also available; without it, there is no use for the wind in the model. Strong crosswinds can greatly penalize A/C performance; however, RWYs are normally designed so that headwind prevails to improve A/C performance. Therefore, such type of winds should be unusual in average.

#### Import file (retrieving only the selected columns)

In [7]:
lcd = pd.read_csv(individualLCLCDileNamePath,
                  encoding='latin1',
                  usecols=cols,
                  low_memory = False)
lcd.sample(5)

,STATION,DATE,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
3667,72517014737,2019-04-13T10:51:00,30.07,71.0,0.00,61.0,FEW:02 28,10.00,MET09904/13/19 10:51:02 METAR KABE 131551Z 240...
6725,72517014737,2019-07-05T07:00:00,NaN,75.0,NaN,79.0,15,8.70,SYN08072517 32464 81704 10239 20200 30076 4021...
8095,72517014737,2019-08-10T08:51:00,29.89,74.0,0.00,57.0,CLR:00,10.00,MET09608/10/19 08:51:02 METAR KABE 101351Z 250...
7027,72517014737,2019-07-12T07:49:00,29.78,77.0,NaN,79.0,BKN:07 17,10.00,MET08207/12/19 07:49:02 SPECI KABE 121249Z 260...
5253,72517014737,2019-05-25T01:00:00,NaN,58.0,NaN,67.0,NaN,9.94,SYN08072517 32966 03503 10144 20083 30076 4021...


## LCD multiple file concatenation (2019)

### Retrieve file list

Let's proceed with multiple-file importing, through concatenation into a single DataFrame.

In [15]:
directory_in_str = os.path.join(root,
                                "Output_Data",
                                "NOAA",
                                "LCD_files")
directory_in_str

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files'

In [17]:
# List the file paths corresponding to each month of year 2019:

file_list = []
try:
    os.listdir(directory_in_str)
except FileNotFoundError:
    print("The system cannot find the specified path:\n" + directory_in_str + "\nPlease check the path has been properly set.")
else:
    for file in os.listdir(directory_in_str):
        if file.endswith(".csv"):
            file_list.append(os.path.join(directory_in_str, file))
            continue
        else:
            continue
file_list    

['C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABE_14737.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABI_13962.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABQ_23050.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABR_14929.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ABY_13869.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ACK_14756.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ACT_13959.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ACV_24283.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ACY_93730.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ADK_25704.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\NOAA\\LCD_files\\ADQ_25501.csv',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Ou

### Generate a DataFrame from concatenation

In [21]:
%%time

# Create a DataFrame from the 12 month-files corresponding to the year 2019

LCD = pd.DataFrame()
for i, csv_path in enumerate(file_list):
    if i == 500: # Fail-safe: in case the list captured more than 500 files (i.e. airports)
        break
    LCD_month = pd.read_csv(csv_path,
                            encoding='latin1',
                            nrows=1e8, # Fail-safe: in case the file is inadvertently too big
                            usecols=cols,
                            low_memory = False) # This will prevent from auto-dtypes
    LCD = LCD.append(LCD_month)
print("Number of LCD files concatenated:", LCD['STATION'].nunique())
LCD

Number of LCD files concatenated: 342
Wall time: 5min 57s


,STATION,DATE,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
0,72517014737,2019-01-01T00:01:00,29.78,43,T,89,OVC:08 5,3.00,MET11101/01/19 00:01:02 SPECI KABE 010501Z 000...
1,72517014737,2019-01-01T00:51:00,29.73,43,0.01,89,OVC:08 5,3.00,MET14401/01/19 00:51:02 METAR KABE 010551Z 000...
2,72517014737,2019-01-01T01:00:00,NaN,43,NaN,89,5,2.98,SYN09272517 17248 80000 10061 20044 39932 4007...
3,72517014737,2019-01-01T01:10:00,29.70,43,NaN,89,OVC:08 5,2.00,MET10501/01/19 01:10:02 SPECI KABE 010610Z 120...
...,...,...,...,...,...,...,...,...,...
18130,70361025339,2019-12-31T22:53:00,29.01,41,0.03,93,OVC:08 13,6.00,MET11012/31/19 22:53:03 METAR PAYA 010753Z 070...
18131,70361025339,2019-12-31T23:53:00,28.97,41,0.03,96,OVC:08 12,3.00,MET13212/31/19 23:53:03 METAR PAYA 010853Z 080...
18132,70361025339,2019-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,SOD73024 HR PRECIPITATION (IN):0.55 24 HR MAX...
18133,70361025339,2019-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,SOM728PCP MTOT:16.90 PCP GT 24HRS:2.22 DATE(S)...


### Save such DataFrame into a CSV file *(only once)*:

In [35]:
output_csv_dir = os.path.join(root,
                              "Output_Data",
                              "NOAA",
                              "LCD_AllStations")
file_name = "LCD_all_preprocessed.csv"

if file_name not in os.listdir(output_csv_dir):
    LCD.to_csv(path_or_buf=output_csv_path,
           index=False,
           encoding='latin1')
else:    
    print("File '" + file_name + "' already exists.\nNo file has been generated (previous one remains).")

File 'LCD_all_preprocessed.csv' already exists.
No file has been generated (previous one remains).


### Explore the data

In [38]:
def val_freq(col='', df=LCD):
    i = 0
    for v in LCD[col].value_counts().sort_index():
        print("{} : {} records ({:.2f}%)" \
              .format(LCD[col].value_counts().sort_index().index[i], v,  v / len(LCD) * 100))
        i += 1

In [39]:
for col in LCD.columns:
    print(col, ':', LCD[col].nunique(), 'unique values')
    if LCD[col].nunique() < 50:
        val_freq(col)
    print("")

STATION : 342 unique values

DATE : 432530 unique values

HourlyAltimeterSetting : 675 unique values

HourlyDryBulbTemperature : 438 unique values

HourlyPrecipitation : 515 unique values

HourlyRelativeHumidity : 196 unique values

HourlySkyConditions : 235226 unique values

HourlyVisibility : 140 unique values

REM : 4481060 unique values



In [40]:
# Absolute & Relative frequency of missing values by column:
pd.set_option('display.max_rows', LCD.shape[1])
missing = pd.DataFrame([LCD.isna().sum(), LCD.isna().sum() / len(LCD) * 100], index=['Absolute', 'Relative']).T.sort_values(by='Relative', ascending=False)
missing

,Absolute,Relative
HourlyPrecipitation,1473764.0,32.769412
HourlyAltimeterSetting,462309.0,10.279525
HourlySkyConditions,417211.0,9.276763
HourlyRelativeHumidity,150503.0,3.346462
HourlyDryBulbTemperature,143158.0,3.183144
HourlyVisibility,141712.0,3.150992
REM,16301.0,0.362456
STATION,0.0,0.000000
DATE,0.0,0.000000


In [41]:
# Quick approach → check how many rows contain empty values to see if directly dropping them would be feasible:

# Check which rows have at least 1 NaN:
LCD[LCD.isna().any(axis=1)]

,STATION,DATE,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
2,72517014737,2019-01-01T01:00:00,NaN,43,NaN,89,5,2.98,SYN09272517 17248 80000 10061 20044 39932 4007...
3,72517014737,2019-01-01T01:10:00,29.70,43,NaN,89,OVC:08 5,2.00,MET10501/01/19 01:10:02 SPECI KABE 010610Z 120...
4,72517014737,2019-01-01T01:17:00,29.69,43,NaN,89,OVC:08 4,2.00,MET10501/01/19 01:17:02 SPECI KABE 010617Z 110...
5,72517014737,2019-01-01T01:44:00,29.65,44,NaN,89,OVC:08 3,1.50,MET10901/01/19 01:44:02 SPECI KABE 010644Z 000...
...,...,...,...,...,...,...,...,...,...
18111,70361025339,2019-12-31T15:00:00,NaN,44,NaN,96,5,6.84,SYN08670361 15261 81105 10067 20061 39884 4989...
18126,70361025339,2019-12-31T21:00:00,NaN,42,NaN,92,8,9.94,SYN08670361 15366 80908 10056 20044 39837 4984...
18132,70361025339,2019-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,SOD73024 HR PRECIPITATION (IN):0.55 24 HR MAX...
18133,70361025339,2019-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,SOM728PCP MTOT:16.90 PCP GT 24HRS:2.22 DATE(S)...


In [36]:
from pandas_profiling import ProfileReport

report_name = file_name[:-4] + '.html'

# Complete report:
prof = ProfileReport(LCD, minimal=True)
prof.to_file(report_name)

# # # Sample report (more computationally efficient)
# prof = ProfileReport(lcd.sample(1000)) 
# prof.to_file(report_name)

### Manipulate the data

In [42]:
LCD.sample(3)

,STATION,DATE,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
7358,72520094823,2019-07-13T07:51:00,30.04,71,0.00,79,CLR:00,10.00,MET09607/13/19 07:51:02 METAR KPIT 131251Z 180...
1868,72745594931,2019-02-11T13:28:00,30.17,19,T,74,BKN:07 21 OVC:08 42,10.00,MET10702/11/19 13:28:02 SPECI KHIB 111928Z 120...
3062,72521014895,2019-03-25T14:51:00,30.08,49,0.00,27,FEW:02 85 FEW:02 160 SCT:04 250,10.00,MET11703/25/19 14:51:02 METAR KCAK 251951Z 020...


#### Date/Time

In [54]:
LCD.insert(loc=1, column='Date', value=LCD['DATE'].str.split(pat='T', n=1, expand=True).iloc[:, 0])
LCD.insert(loc=2, column='Time', value=LCD['DATE'].str.split(pat='T', n=1, expand=True).iloc[:, 1])
LCD.insert(loc=3, column='Time_h', value=LCD['Time'].str.split(pat=':', expand=True).iloc[:, 0])
LCD.drop(['DATE'], axis=1, inplace=True)
LCD.sample(3)

,STATION,Date,Time,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
11352,72503014732,2019-10-24,13:00:00,NaN,66,NaN,25,NaN,9.94,SYN08072503 32966 02207 10189 21017 30237 4024...
17940,91212041415,2019-12-27,01:54:00,29.9,76,0.00,79,FEW:02 60,10.00,MET10012/27/19 01:54:02 METAR PGUM 261554Z 050...
2898,72440013995,2019-03-11,13:50:00,30.30,57,NaN,51,BKN:07 25 OVC:08 65,10.00,MET08803/11/19 13:50:02 SPECI KSGF 111950Z 000...


In [58]:
LCD.insert(loc=3, column='Time_h', value=LCD['Time'].str.split(pat=':', expand=True).iloc[:, 0])

In [59]:
LCD.sample(3)

,STATION,Date,Time,Time_h,HourlyAltimeterSetting,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,REM
3468,72226013895,2019-03-23,09:53:00,09,30.30,64,0.00,34,CLR:00,10.00,MET09603/23/19 09:53:02 METAR KMGM 231553Z ...
2673,72483993225,2019-04-01,23:53:00,23,29.87,55,0.01,90,BKN:07 50 BKN:07 70 OVC:08 85,10.00,MET13304/01/19 23:53:02 METAR KSMF 020753Z 110...
11991,91212041415,2019-09-09,20:54:00,20,29.71,77,T,94,OVC:08 80,10.00,MET11109/09/19 20:54:01 METAR PGUM 091054Z 020...


# Antes de continuar, hacer un merge con OTP-WBAN para ver si se cubren suficientes vuelos considerando estación, hora de la medición, etc.

In [ ]:
t1 = time.perf_counter() - t0
print("Time elapsed: ", t1) # CPU seconds elapsed (floating point)

___

___